In [9]:
#Right now we could maybe widdle down to movies above a certain rating + number of ratings

In [46]:
import pandas as pd
import csv
import time
import numpy as np
import itertools

In [47]:
df_ratings = pd.read_csv('data/ratings.tsv',sep = '\t')
titles = pd.read_csv('data/titles.tsv',sep='\t')
names = pd.read_csv('data/names.tsv',sep = '\t')
movies = titles[titles['titleType']=='movie']
movies = list(movies['tconst'].unique())

C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(len(movies))

573966


In [69]:
print(names[names['primaryName']=='Sean Bean'])
df_ratings.head()
np.mean(df_ratings['numVotes'])
np.std(df_ratings['averageRating'])

             nconst primaryName birthYear deathYear  \
292       nm0000293   Sean Bean      1959        \N   
2777559  nm11881915   Sean Bean        \N        \N   
3122961  nm12232472   Sean Bean        \N        \N   

                           primaryProfession  \
292      actor,producer,animation_department   
2777559                             producer   
3122961                      cinematographer   

                                  knownForTitles  
292      tt1181791,tt0120737,tt0167261,tt0944947  
2777559                                tt2107721  
3122961                                tt0852584  


1.3978072280174874

In [76]:
##get a list of movies based on a rating threshold and year threshold
def filterMovies(ratingThreshold,yearThreshold,dfratings,dftitles):
    avgVotes = np.mean(df_ratings['numVotes']) + 100000
    qualifiedMovies = []
    len(df_ratings)
    boolean_series = dfratings.tconst.isin(movies)
    dfratings = dfratings[boolean_series]
    for row in dfratings.iterrows():
        if row[1]['numVotes'] >= avgVotes:
            if row[1]['averageRating'] >= ratingThreshold:
                qualifiedMovies.append(row[1]['tconst'])
    qualifiedMoviesFinal = []
    for movie in qualifiedMovies:
        year = (list(dftitles[dftitles['tconst']==movie]['startYear'])[0])
        if type(year) == str:
            year = int(year)
        if year >= yearThreshold:
            qualifiedMoviesFinal.append(movie)
    return qualifiedMoviesFinal

In [62]:
print(len(qualifiedMovies))

2468


In [80]:
qualifiedMovies = filterMovies(4.5,2000,df_ratings,titles)
print(len(qualifiedMovies))

1411


In [81]:
## find the frequent actors in the list of movies that meets the support threshold
def findFreqActors(qMovies,supportThreshold):
    count = 0
    freqActors = dict()
    for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
        chunk = chunk[chunk['category']=='actor']
        chunk = chunk.groupby('tconst')
        for name, group in chunk:
            if name not in qMovies:
                continue
            else:
                actors = group['nconst'].unique()
                for actor in actors:
                    if actor in freqActors:
                        freqActors[actor][0] += 1
                        if name in qMovies:
                            freqActors[actor][1].append(name)
                    else:
                        freqActors[actor] = [1,[name]]
    freqActorsFinal = dict()
    for actor in freqActors:
        if freqActors[actor][0] >= supportThreshold:
            freqActorsFinal[actor] = freqActors[actor]
    return freqActors

In [82]:
freqActors = findFreqActors(qualifiedMovies,2)

In [ ]:
###### print(len(freqActors))

In [24]:
print(freqActors['nm0004051'])
print(freqActors['nm0000293'])

[2, ['tt0326306', 'tt0372183']]
[2, ['tt0120737', 'tt5198068']]


In [83]:
print(len(freqActors))

1392


In [84]:
def findPairs():
    freq_pairs = dict()
    for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
        chunk = chunk[chunk['category']=='actor']
        chunk = chunk.groupby('tconst')
        for name, group in chunk:
            #print(name)
            if name not in qualifiedMovies:
                continue
            else:
                freqActorsGroup = [actor for actor in group['nconst'] if actor in freqActors]
                freqPairsGroup = list(itertools.combinations(freqActorsGroup,2))
                for pair in freqPairsGroup:
                    if pair in freq_pairs:
                        freq_pairs[pair][0] += 1
                        freq_pairs[pair][1].append(name)
                    else:
                        freq_pairs[pair] = [1,[name]]
        freq_pairs = dict(sorted(freq_pairs.items(), key=lambda item: item[1],reverse=True))
        freq_pairsFinal = dict()
        for pair in freq_pairs:
            if (pair[1],pair[0]) not in freq_pairsFinal:
                freq_pairsFinal[pair] = freq_pairs[pair]
    return freq_pairsFinal

In [ ]:
for actor in freqActors:
    for actor2 in freqActors:
        if actor == actor2 or (actor2,actor) in freqPairs:
            continue:
        else:
            movies_together = []
            for movie in freqActors[actor][1]:
                if movie in freqActors[actor2][1]:
                    movies_together.append(movie)
            for movie in freqActors[actor2][1]:
                if movie in freqActors[actor][1] and movie not in movies_together:
                    movies_together.append(movie)
            freqPairs[(actor,actor2)] = [len(movies_together),movies_together]
        

In [85]:
freq_pairs = findPairs()

In [86]:
print(len(freq_pairs))
freqpairsfinal = dict()
for pair in freq_pairs:
    if (freq_pairs[pair][0] >= 2):
        freqpairsfinal[pair] = freq_pairs[pair]
    #print(getName(pair[0]),getName(pair[1]))
    

3257


In [87]:
print(len(freqpairsfinal))

162


In [88]:
def getName(nconst):
    return list(names[names['nconst']==nconst]['primaryName'])[0] 
def getSoloMovies(name1,name2,pairsDict,soloDict):
    name1solomovies = []
    name2solomovies = []
    for movie in soloDict[name1][1]:
        if movie not in pairsDict[(name1,name2)][1]:
            name1solomovies.append(movie)
    for movie in soloDict[name2][1]:
        if movie not in pairsDict[(name1,name2)][1]:
            name2solomovies.append(movie)
    for movie in name1solomovies:
        if movie in name2solomovies:
            pairsDict[(name1,name2)][1].append(movie)
            name1solomovies.remove(movie)
            name2solomovies.remove(movie)
    return name1solomovies,name2solomovies
def getRatings(movies):
    ratings = []
    if len(movies) == 0:
        return 0
    for movie in movies:
        if len(list(df_ratings[df_ratings['tconst']== movie]['averageRating']))>0:
            ratings.append(list(df_ratings[df_ratings['tconst']== movie]['averageRating'])[0])
    rating = np.mean(ratings)
    return rating

In [ ]:
for pair in freqpairsfinal:
    movies1, movies2 = getSoloMovies(pair[0],pair[1],freqpairsfinal,freqActors)
    ratings1 = getRatings(movies1)
    ratings2 = getRatings(movies2)
    ratings_together = getRatings(freqpairsfinal[pair][1])
    betteralone1 = True
    betteralone2 = True
    bt1 = False
    bt2 = False
    bettertogether = False
    if ratings_together >= ratings1:
        betteralone1 = False
        bt1 = True
    if ratings_together >= ratings2:
        betteralone2 = False
        bt2 = True
    if bt1 or bt2:
        bettertogether = True
    if bettertogether and ratings1 != 0 and ratings2 != 0:
        #print(movies1)
        #print(movies2)
        #print(freqpairsfinal[pair][1])
        print("For the duo " + getName(pair[0]) + " , " + getName(pair[1]) + "Their rating of " + str(ratings_together) + "was betteer than both of their solo ratings: " + str(ratings1) + ", " + str(ratings2))


For the duo Adam Sandler , Kevin JamesTheir rating of 6.083333333333333was betteer than both of their solo ratings: 6.425, 5.9
For the duo Vin Diesel , Paul WalkerTheir rating of 6.959999999999999was betteer than both of their solo ratings: 6.9142857142857155, 5.9
For the duo Vin Diesel , Dwayne JohnsonTheir rating of 7.0249999999999995was betteer than both of their solo ratings: 6.8875, 6.223529411764705
For the duo Josh Hutcherson , Liam HemsworthTheir rating of 6.975was betteer than both of their solo ratings: 6.449999999999999, 5.9
For the duo Robert Downey Jr. , Chris EvansTheir rating of 7.9799999999999995was betteer than both of their solo ratings: 7.2, 6.860000000000001
For the duo Robert Downey Jr. , Mark RuffaloTheir rating of 7.949999999999999was betteer than both of their solo ratings: 7.260000000000001, 7.3100000000000005
For the duo Simon Pegg , Nick FrostTheir rating of 7.425was betteer than both of their solo ratings: 7.540000000000001, 7.4
For the duo Johnny Depp , Geo

In [47]:
titles[titles['tconst']=='tt0108108']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
105755,tt0108108,tvEpisode,Sharpe's Rifles,Sharpe's Rifles,0,1993,\N,102,Adventure


C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
#HAIL MARY
def findAllMovies(nconst):
    allMovies = []
    for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
        chunk = chunk[chunk['category']=='actor']
        chunk = chunk.groupby('nconst')
        for name, group in chunk:
            allMovies+=list(group['tconst'])
    return allMovies
findAllMovies('nm0004051')

KeyboardInterrupt: 